In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle

In [2]:
mnist_dataset = fetch_openml('mnist_784',version=1)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/datasets/_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:
X = mnist_dataset.data
print(X[:5])

   pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  pixel9  \
0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
1     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
2     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
3     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   
4     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0     0.0   

   pixel10  ...  pixel775  pixel776  pixel777  pixel778  pixel779  pixel780  \
0      0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
1      0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
2      0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
3      0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   
4      0.0  ...       0.0       0.0       0.0       0.0       0.0       0.0   

   pixel781  pixel782  pixel783  pixel784  
0       0.0       

In [4]:
y = mnist_dataset.target
print(y[:5])

0    5
1    0
2    4
3    1
4    9
Name: class, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']


In [5]:
#split train and test set
num_of_train_images = 60000
num_of_test_images = 10000
X_train = X[:num_of_train_images]
X_test = X[num_of_train_images:]
y_train = y[:num_of_train_images]
y_test = y[num_of_train_images:]

In [6]:
knnC = KNeighborsClassifier() #5 neighbours by default
knnC.fit(X_train,y_train)
yhat_knn = knnC.predict(X_test)

In [7]:
print("accuracy with KNeighboursClassifier")
print(accuracy_score(y_test,yhat_knn))
print("f1 score with KNeighboursClassifier")
print(f1_score(y_test,yhat_knn,average="weighted"))

accuracy with KNeighboursClassifier
0.9688
f1 score with KNeighboursClassifier
0.9687470572168784


In [8]:
grid_parameters = {'weights':["uniform","distance"],
                    'n_neighbors':[3,5]}

grid_search = GridSearchCV(knnC,param_grid=grid_parameters,cv=5,scoring="accuracy")
grid_search.fit(X_train,y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy')

In [9]:
grid_search.best_estimator_

KNeighborsClassifier(n_neighbors=3, weights='distance')

In [10]:
grid_search.best_params_

{'n_neighbors': 3, 'weights': 'distance'}

In [11]:
grid_search.best_score_

0.9711166666666665

In [12]:
yhat_grid = grid_search.predict(X_test)
print("accuracy with GridSearchCV and KNeighborsClassifier")
print(accuracy_score(y_test,yhat_grid))

accuracy with GridSearchCV and KNeighborsClassifier
0.9717


In [13]:
y.head(11)

0     5
1     0
2     4
3     1
4     9
5     2
6     1
7     3
8     1
9     4
10    3
Name: class, dtype: category
Categories (10, object): ['0', '1', '2', '3', ..., '6', '7', '8', '9']

In [14]:
def shift_image_by_a_pixel(x_data,index_names):
    X_mod = np.array(x_data).reshape(28,28)
    X_train_right = pd.DataFrame(np.roll(X_mod,1).reshape(1,784),columns=index_names)
    X_train_left = pd.DataFrame(np.roll(X_mod,-1).reshape(1,784),columns=index_names)
    X_train_up = pd.DataFrame(np.roll(X_mod,-1,axis=0).reshape(1,784),columns=index_names)
    X_train_down = pd.DataFrame(np.roll(X_mod,1,axis=0).reshape(1,784),columns=index_names)
    X_mod = pd.DataFrame(x_data,columns=index_names)
    df = pd.concat([X_train_right,X_train_left,X_train_up,X_train_down])
    return df

In [15]:
num_of_shifted_images = 1000
X_shifted = X_train[:num_of_shifted_images].copy()
y_shifted= y_train[:num_of_shifted_images].copy()

for i, image in X_shifted.iterrows():
    image_shifted_df = shift_image_by_a_pixel(X_shifted.iloc[i],X_shifted.columns.to_list())
    X_shifted = pd.concat([X_shifted,image_shifted_df])
    #actual values of the 4 rotated images
    y_new_rotated = pd.DataFrame([y_train.iloc[i],y_train.iloc[i],y_train.iloc[i],y_train.iloc[i]])
    y_shifted = pd.concat([y_shifted,y_new_rotated])

In [16]:
knnC_2 = KNeighborsClassifier(n_neighbors=3,weights="distance")
knnC_2.fit(X_shifted,y_shifted)
y_hat_2 = knnC_2.predict(X_test)
print("accuracy with shifted set added to the training set")
print(accuracy_score(y_hat_2,y_test))

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


accuracy with shifted set added to the training set
0.8898
